# Solutions IV: Validation and Selection

In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, KFold

from sklearn.metrics import accuracy_score

## Load Data

In [ ]:
# Read in the banking data set.
df = pd.read_csv("../../0_data/banking/bank-additional-full.csv", sep=";")
df.head(3)

In [ ]:
# Split features and labels.
X = df.drop(columns=["duration", "pdays", "y"])
y = df["y"]

## Create Models

In [ ]:
# Get column names per data type.
categorical = X.select_dtypes("object").columns
numerical = X.select_dtypes("number").columns

In [ ]:
# Set up the data preparation.
transformer = ColumnTransformer(
    transformers=[
        ("encode_categorical", OneHotEncoder(handle_unknown="ignore"), categorical),
    ],
    remainder="passthrough",
)

In [ ]:
# Create the DummyClassifier.
dummy_model = DummyClassifier(strategy="stratified")

In [ ]:
# Create pipeline for the RandomForestClassifier.
forest_model = Pipeline(
    steps=[
        ("preparation", transformer),
        ("model", RandomForestClassifier(n_jobs=-1)),
    ]
)

## Train versus Test Performance

In [ ]:
# Split X and y into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7)

In [ ]:
# Check sizes
print("Total records: ", X.shape[0])
print("Train records: ", X_train.shape[0])
print("Test  records: ", X_test.shape[0])

In [ ]:
# Get performance of DummyClassifier and RandomForestClassifier.
# Report performance on both train and test sets.
models = {"Dummy": dummy_model, "RandomForest": forest_model}

print("-" * 40)
for model_name, model in models.items():
    model.fit(X_train, y_train)

    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)

    # Compute accuracy
    train_acc = accuracy_score(y_train, predict_train)
    test_acc = accuracy_score(y_test, predict_test)

    # Print report
    print(f"Model:              {model_name:>20s}")
    print(f"Accuracy train:     {train_acc:20.3f}")
    print(f"Accuracy test:      {test_acc:20.3f}")
    print("-" * 40)

In [ ]:
# Note: The RandomForestClassifier seems to overfit quite a bit.

In [ ]:
# Does DummyClassifier outperform always predicting no?
# It does not, because it draws at random from the labels yes / no.
print("Accuracy Always No")
print(f"Accuracy train: {(y_train == 'no').sum() / y_train.shape[0]:.3f}")
print(f"Accuracy test:  {(y_test == 'no').sum()  / y_test.shape[0]:.3f}")

## Cross Validation

In [ ]:
# Create KFold object and specify number of splits.
kfold = KFold(n_splits=5)

In [ ]:
# Perform cross-validation with 5 splits.
kfold = KFold(n_splits=5)
for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y), start=1):

    # Create data sets for the fold.
    # Note: Use .iloc[] because we get indices!
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]

    X_test = X.iloc[test_idx]
    y_test = y.iloc[test_idx]

    # Train the model, get predictions and accuracy.
    forest_model.fit(X_train, y_train)
    predictions = forest_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    # Print details.
    print(f"----------- [Fold {fold}] ----------")
    print(f"Test cases:       {test_idx[0]:5d} - {test_idx[-1]:5d}")
    print(f"Accuracy:         {accuracy:13.3f}")

In [ ]:
# Plot occurrence of the "yes" label throughout the dataset.
(
    pd.DataFrame({
        "index": df.index,
        "total_yes": (df["y"] == "yes").cumsum(),
    })
    .plot(
        x="index",
        y="total_yes",
        title="Label: Yes - Cumulative",
        figsize=(10, 3),
    )
)
None

In [ ]:
# Plot dispersion of the months throughout the dataset.
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
ax = (
    pd.DataFrame({
        "index": df.index,
        "month_num": df["month"].map(lambda m: months.index(m) + 1)
    })
    .plot(
        x="index",
        y="month_num",
        title="Months",
        figsize=(10, 3),
        legend=False,
    )
)
ax.set_yticks([m + 1 for m in range(12)], months)
ax.grid(visible=True, color="lightgrey", axis="y")
None

In [ ]:
# Plot how price index changes throughout the dataset.
ax = (
    pd.DataFrame({
        "index": df.index,
        "price_index": df["cons.price.idx"]
    })
    .plot(
        x="index",
        y="price_index",
        title="Price Ixdex",
        figsize=(10, 3),
        legend=False,
    )
)
None